In [1]:
import torch
import torch.nn as nn
import torch.utils.data as data
import os
import numpy as np
import json

In [2]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

In [3]:
class JsonDataset(data.Dataset):
    def __init__(self, data_path):
        f = open(data_path, 'r')
        self.data = json.loads(f.read())
        f.close()

    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        return torch.FloatTensor(self.data[index][0]), \
            torch.LongTensor([float(self.data[index][1])])
                # torch.FloatTensor(self.data[index][0]).sum()
        #if self.data[index][0][-1] > 0.1:
        #    return torch.FloatTensor(self.data[index][0]), torch.LongTensor([1])
        #else:
        #    return torch.FloatTensor(self.data[index][0]), torch.LongTensor([0])
            
            

In [5]:
train_data = JsonDataset('naive-train.json')
validation_data = JsonDataset('naive-validation.json')
test_data = JsonDataset('naive-test.json')

In [6]:
input_size = train_data[0][0].shape[0]
batch_size = 64
fc_hidden_size = 100
fc_num_layers = 5

In [7]:
params = {'batch_size': batch_size, 'shuffle': True, 'num_workers': 1, 'pin_memory': True}
train_loader = data.DataLoader(train_data, **params)
validation_loader = data.DataLoader(validation_data, **params)
test_loader = data.DataLoader(test_data, **params)

In [14]:
class Naive(nn.Module):
    def __init__(self, input_size, fc_hidden_size, fc_num_layers):
        super(Naive, self).__init__()

        seq = []
        seq.append(nn.Linear(input_size, fc_hidden_size))
        seq.append(nn.Tanh())

        for _ in range(fc_num_layers - 1):
            seq.append(nn.Linear(fc_hidden_size, fc_hidden_size))
            seq.append(nn.Tanh())
            #seq.append(nn.Sigmoid())
            seq.append(nn.Dropout(0.5))
        
        seq.append(nn.Linear(fc_hidden_size, 2))

        self.fc = nn.Sequential(*seq)

    def forward(self, x):
        out = self.fc(x)
        
        return out

In [15]:
def train(model, criterion, optimizer):
    model.train()
    train_loss = 0
    correct = 0
    total = 0

    for i, (inputs, targets) in enumerate(train_loader):
        inputs = inputs.to(device)
        targets = torch.flatten(targets).to(device)
        #targets = targets.reshape(-1, 1).to(device)

        outputs = model(inputs)
        loss = criterion(outputs, targets)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total += targets.size(0)
        train_loss += loss.item() * targets.size(0)
        _, predicted = outputs.max(1)
        correct += predicted.eq(targets).sum().item()
        
    epoch_train_loss = train_loss / total
    epoch_train_acc = float(100 * correct / total)

    return epoch_train_loss, epoch_train_acc

In [16]:
def validation(model, criterion):
    model.eval()
    validation_loss = 0
    correct = 0
    total = 0

    with torch.no_grad():
        for i, (inputs, targets) in enumerate(validation_loader):
            inputs = inputs.to(device)
            targets = torch.flatten(targets).to(device)
            #targets = targets.reshape(-1, 1).to(device)

            outputs = model(inputs)
            loss = criterion(outputs, targets)

        total += targets.size(0)
        validation_loss += loss.item() * targets.size(0)
        _, predicted = outputs.max(1)
        correct += predicted.eq(targets).sum().item()
        
    epoch_validation_loss = validation_loss / total
    epoch_validation_acc = float(100 * correct / total)

    return epoch_validation_loss, epoch_validation_acc

In [17]:
weight_zero = len([i for i in range(len(train_data)) if train_data[i][1] == 1]) / len(train_data)
print('weight_zero: {}'.format(weight_zero))

model = Naive(input_size, fc_hidden_size, fc_num_layers).to(device)
criterion = nn.CrossEntropyLoss(weight=torch.tensor([weight_zero, 1 - weight_zero]).to(device))
#criterion = nn.CrossEntropyLoss().to(device)
#criterion = nn.MSELoss().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)  

num_epochs = 100

weight_zero: 0.49171875


In [18]:
best_validation_loss = None

for epoch in range(0, num_epochs):
    epoch_train_loss, epoch_train_acc = train(model, criterion, optimizer)
    epoch_validation_loss, epoch_validation_acc = validation(
                                                    model, criterion)
    
    if best_validation_loss == None or epoch_validation_loss < best_validation_loss:
        torch.save(model.state_dict(), 'best_naive.pth')
        print('Saved.')
        best_validation_loss = epoch_validation_loss

    #epoch_test_loss, epoch_test_acc = test(net, criterion, vgg['best_acc'], 'vgg_best.pth')
    #vgg['test_loss'].append(epoch_test_loss)
    #vgg['test_acc'].append(epoch_test_acc)
    #if epoch_test_acc > vgg['best_acc']:
    #    vgg['best_acc'] = epoch_test_acc

    print('Epoch {}. Training loss: {} ({}% accuracy). Validation loss: {} ({}% accuracy)'
        .format(epoch + 1, 
                format(epoch_train_loss, '.4f'), format(epoch_train_acc, '.4f'),
                format(epoch_validation_loss, '.4f'), format(epoch_validation_acc, '.4f')))
    
#print('Best test accuracy: {}%'.format(vgg['best_acc']))

Saved.
Epoch 1. Training loss: 0.6939 (50.4219% accuracy). Validation loss: 0.6931 (48.4375% accuracy)
Epoch 2. Training loss: 0.6940 (48.5781% accuracy). Validation loss: 0.6934 (50.0000% accuracy)
Saved.
Epoch 3. Training loss: 0.6929 (51.8125% accuracy). Validation loss: 0.6886 (54.6875% accuracy)
Epoch 4. Training loss: 0.6847 (53.6875% accuracy). Validation loss: 0.7157 (56.2500% accuracy)
Saved.
Epoch 5. Training loss: 0.6308 (63.2031% accuracy). Validation loss: 0.6506 (64.0625% accuracy)
Saved.
Epoch 6. Training loss: 0.5964 (66.9688% accuracy). Validation loss: 0.5849 (73.4375% accuracy)
Saved.
Epoch 7. Training loss: 0.5738 (69.5000% accuracy). Validation loss: 0.4618 (82.8125% accuracy)
Epoch 8. Training loss: 0.5526 (71.6875% accuracy). Validation loss: 0.5458 (73.4375% accuracy)
Saved.
Epoch 9. Training loss: 0.5423 (73.1406% accuracy). Validation loss: 0.4200 (84.3750% accuracy)
Epoch 10. Training loss: 0.5334 (73.6562% accuracy). Validation loss: 0.4576 (79.6875% accurac

In [ ]:
#print(len(model.fc))
#model.fc[10].bias

In [19]:
# Test
model.load_state_dict(torch.load('best_naive.pth'))

with torch.no_grad():
    n_correct = 0
    n_samples = 0
    for i, (inputs, targets) in enumerate(test_loader):
        inputs = inputs.to(device)
        targets = torch.flatten(targets).to(device)
        #targets = targets.reshape(-1, 1).to(device)

        outputs = model(inputs)
        _, predicted = torch.max(outputs.data, 1)
        #if i == 0:
            #print(outputs)
            #print(predicted)
            #print(targets)

        n_samples += inputs.shape[0]
        n_correct += (predicted == targets).sum().item()
    
    acc = float(100 * n_correct / n_samples)
    print('Test accuracy: {}%'.format(acc))

Test accuracy: 78.3%


In [ ]:
"""
# Train
for epoch in range(num_epochs):
    for i, (inputs, targets) in enumerate(train_loader): 
        model.train()

        inputs = inputs.to(device)
        #targets = torch.flatten(targets).to(device)
        targets = targets.reshape(-1, 1).to(device)
        
        # Forward pass
        outputs = model(inputs)
        loss = criterion(outputs, targets)
        
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    
    print('Epoch {}. Loss: {}'.format(epoch, loss.item()))
"""

Epoch 0. Loss: 0.8501914143562317
Epoch 1. Loss: 0.2044791579246521
Epoch 2. Loss: 0.1138804629445076
Epoch 3. Loss: 0.0778626948595047
Epoch 4. Loss: 0.04106287285685539
Epoch 5. Loss: 0.02865191176533699
Epoch 6. Loss: 0.03518471121788025
Epoch 7. Loss: 0.005256733391433954
Epoch 8. Loss: 0.0062057580798864365
Epoch 9. Loss: 0.002033299533650279
Epoch 10. Loss: 0.001143027562648058
Epoch 11. Loss: 0.00204146234318614
Epoch 12. Loss: 0.00045666348887607455
Epoch 13. Loss: 0.013698620721697807
Epoch 14. Loss: 0.0010930895805358887
Epoch 15. Loss: 0.009303990751504898
Epoch 16. Loss: 0.015400063246488571
Epoch 17. Loss: 0.009674233384430408
Epoch 18. Loss: 0.013845495879650116
Epoch 19. Loss: 0.002136755967512727


In [ ]:
# Test
with torch.no_grad():
    n_correct = 0
    n_samples = 0
    for i, (inputs, targets) in enumerate(test_loader):
        inputs = inputs.to(device)
        targets = torch.flatten(targets).to(device)

        outputs = model(inputs)
        _, predicted = torch.max(outputs.data, 1)
        if i == 0:
            print(predicted)

        n_samples += inputs.shape[0]
        n_correct += (predicted == targets).sum().item()
    
    accu = 100.0 * n_correct / n_samples
    print('Accuracy: {}%'.format(100 * n_correct / n_samples))

tensor([1, 1, 0, 1, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0,
        1, 0, 0, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 0, 0,
        1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 0], device='cuda:0')
Accuracy: 49.5%


In [ ]:
f = open('naive-train.json', 'r')
a = json.loads(f.read())

In [ ]:
a = torch.randn(3, 5, requires_grad=True)
a


tensor([[-1.0103,  0.2698, -0.6094, -2.6507,  0.7647],
        [-0.3342, -0.4605, -0.7460,  1.0389, -1.7225],
        [-1.3920, -2.3970,  1.5567, -0.0712,  0.9943]], requires_grad=True)

In [ ]:
torch.flatten(torch.zeros(3, 1)).shape

torch.Size([3])